# Delayed

> Classes to delay execution of functions and methods.

In [1]:
#| default_exp core
#| hide
#| eval: false
%reload_ext autoreload
%autoreload 2

In [7]:
#| export
import inspect

from functools import partial
from fastcore.basics import *
from fastcore.meta import *
from typing import Union
from pikaQ.utils import *

try: from types import UnionType
except ImportError: UnionType = None

In [8]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
# allow multiple output from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

We can easily delay the execution of a function by storing the function definition, its arguments and keyword arguments in a `DelayedFunc` object.
The `exec` method on the object can then be called later to execute the function.
During the execution of the function, we can provide keyword arguments to override the ones that were provided when the `DelayedFunc` object was created.

In [12]:
#| export
class DelayedFunc:
    """Delay the execution of stored function until exec is run."""
    def __init__(self, func, args, kwargs, order=None) -> None: 
        store_attr()

    def exec(self, func=None, **kwargs):
        """keyword arguments can be overwritten with any provided new kwargs."""
        self.kwargs.update(kwargs)
        # recursively resolve all delayed functions
        args = (exec(arg, func, **self.kwargs) for arg in self.args) 
        return self.func(*args, **self.kwargs)


class DelayedTerm(DelayedFunc):
    """Delay the generation of sql terms until get_sql is run."""
    def __init__(self, func, args, kwargs, order=None) -> None:
        super().__init__(func, args, kwargs, order)

    def get_sql(self, **kwargs):
        return self.exec(func=str, **kwargs)

In [17]:
def multiply(x, multiplier=2):
    return x * multiplier


def add_months(column, num, dialect='sql'):
    if dialect=='sql':
        return f'DATE_ADD(month, {num}, {column})'
    elif dialect=='snowflake':
        return f'ADD_MONTHS({column}, {num})'


def to_date(date, format=None, dialect='sql'):
    if dialect in ('sql', 'snowflake'):
        if format is None:
            return f"TO_DATE('{date}')"
        else:
            return f"TO_DATE('{date}', '{format}')"
    

test_eq(DelayedFunc(multiply, (2,), {'multiplier': 3}).exec(multiplier=4), 8)

dl_to_date = DelayedTerm(to_date, ('2020-01-01',), {})
dl_add_months = DelayedTerm(add_months, (dl_to_date, 1), {})

test_eq(
    dl_add_months.get_sql(dialect='snowflake'),
    "ADD_MONTHS(TO_DATE('2020-01-01'), 1)")
test_eq(
    dl_add_months.get_sql(dialect='sql'),
    "DATE_ADD(month, 1, TO_DATE('2020-01-01'))")



However, this is not enough for our purpose. We also need the ability to delay all instance methods until `.exec` is called. To expand this functionality, we introduce `DelayedPipeline` and the decorator `@delayed_methods`.

In [18]:
#|hide
import nbdev; nbdev.nbdev_export()